# Onboard a model defined in a .wf1 file
This notebook takes a .wf1 workfile and transform  it to a modelflow model.<br>

The overall process is performed in a Dataclass named:**GrapWfModel**<br> 
Close study of this class can be rewarding, but is outside the need of most users.<br>
The overall structure is: 
 1. Eviews is started and the wf1 file is loaded
 2. 1.    Some transformations are performed on data.
 3. 2.   The model is unlinked 
 4. 3.    The workspace is saved as a wf2 file. Same name with ```_modelflow appended```
 5. Eviews is closed 
 6. The wf2 file is read as a json file. 
 7. Relevant objects are extracted. 
 7. The MFMSA variable is  extracted, to be saved in the dumpfile. 
 8. The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 9. Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)  
 9. For Stochastic equations new fitted variables are generated - without add add_factors and dummies.  
 9. A model to generate fitted variables is created  
 9. A model to generate add_factors is created. 
 9. A model encompassing the original equations, the model for fitted variables and for add_factors is created. 
 9. The data series and scalars are shoveled into a Pandas dataframe 
 11. 1. Some special series are generated as the expression can not be incorporated into modelflow model specifications
 11. 2. The model for fitted values is simulated in the specified timespan
 12. 3. The model for add_factors is simulated in the timespan set in MFMSA
 10. The data descriptions are extracted into a dictionary. 
 10. 1. Data descriptions for dummies, fixed values, fitted values and add_factors are derived. 
 11. Now we have a model and a dataframe with all variables which are needed.
 b
The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  


## Prerequisites  

- Eviews version 12 
- The python library: ```pyevies```  


In [5]:
from pathlib import Path

from modelclass import model 
from modelgrabwf2 import GrabWfModel
model.widescreen()
model.scroll_off()

In [6]:
 %load_ext autoreload
 %autoreload 2

## Model specific transformations 
Not all Eviews equations have a direct counterpart in modelflow.<br>
To handle that two types of transformations can be provided for a specific model. 
- **eviews_run_lines** which specifies a list of Eviews command lines to be run. Typical to make some transformation of data which is not possible in modelflow. 
- **country_trans** which specifies a list of replacements to be done to the Eviews formulas before they are processed further. Typical to use the transformation mentioned above.  

In [7]:
ago_trans = lambda  input : input.replace('@MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")','MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD') 
ago_eviews_run_lines = ['smpl @ALL','series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")']

## Process the model

In [8]:
all_about_ago = GrabWfModel(r'wfs\agosoln.wf1', 
                    eviews_run_lines= ago_eviews_run_lines,
                    country_trans    =  ago_trans,
                    make_fitted = True,        # make equatios for fitted values of stocastic equations 
                    do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                    fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                    fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                    disable_progress =True) 


Reading c:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews\wfs\agosoln.wf1
Assummed model name: AGO
Eviewsline to run :smpl @ALL
Eviewsline to run :series MEAN_AGOBNCABFUNDCD_DIV_AGONYGDPMKTPCD = @MEAN(AGOBNCABFUNDCD/AGONYGDPMKTPCD,"2000 2020")
The model: AGO is unlinked 
Writing C:\wb new\Modelflow\working_paper\thebook\content\howto\onboard\eviews\wfs\agosoln_modelflow.wf2
Model name: AGO

Processing the model:AGO


Normalizing model: 100%|███████████████████████████████████████████████████████████████████████████████████████|198/198


testmodel solved  
Calculation of add factors for AGO solved  


## Check if each equation on its own result in the values provided. 
aka: residual check <br> 
If they are not pretty close, something is very wrong. 

In [9]:
all_about_ago.test_model(all_about_ago.start,all_about_ago.end,maxerr=100,tol=100,showall=0)   # tol determins the max acceptable absolute difference 

AGO solved  
testmodel solved  

Chekking residuals for AGO 2016 to 2060

Variable with residuals above threshold
AGONECONPRVTCN                        , Max difference:  2861.03139496 Max Pct    0.0000106457% It is number    45 in the solveorder and error number 1
AGONEGDIFTOTCN                        , Max difference:   177.46922302 Max Pct    0.0000007316% It is number    59 in the solveorder and error number 2
AGONEEXPGOILCN                        , Max difference:   544.94689941 Max Pct    0.0000026926% It is number    68 in the solveorder and error number 3
AGONEEXPGNFSCN                        , Max difference:   504.25928497 Max Pct    0.0000023486% It is number    69 in the solveorder and error number 4
AGONEIMPGNFSCN                        , Max difference:  1101.58077240 Max Pct    0.0000111348% It is number    73 in the solveorder and error number 5
AGONYYWBTOTLCN                        , Max difference:   286.10862160 Max Pct    0.0000091250% It is number    82 in the solv

The absolute differences are surprising large, however it seems they reflect some shortcut in the eviews calculation.

## Extract the model and the baseline
**all_about_ago** has a lot of content including. 
- .mmodel is the model instance
- .base_input is the baseline where the add factors and the fitted values are calculated 

In [10]:
mago    = all_about_ago.mmodel       # the model instance  
baseline = all_about_ago.base_input

## Run the model 
In order to achieve numerical stability Gauss-Seidle has to be dampened: alfa=0.7 makes sure that the solution does not explode. 
The convergence criteria is tightend a lot. 

In [11]:
res = mago(all_about_ago.base_input,2016,2040,silent=0,alfa=0.7,ldumpvar=1)

Will start solving: AGO
Create compiled solving function for AGO
ljit=False stringjit=True  transpile_reset=False  hasattr(self, f"pro_{jitname}")=False
now makelos makes a sim solvefunction
2016 Solved in 6 iterations
2017 Solved in 6 iterations
2018 Solved in 6 iterations
2019 Solved in 6 iterations
2020 Solved in 7 iterations
2021 Solved in 7 iterations
2022 Solved in 7 iterations
2023 Solved in 7 iterations
2024 Solved in 8 iterations
2025 Solved in 7 iterations
2026 Solved in 21 iterations
2027 Solved in 7 iterations
2028 Solved in 8 iterations
2029 Solved in 7 iterations
2030 Solved in 12 iterations
2031 Solved in 12 iterations
2032 Solved in 12 iterations
2033 Solved in 13 iterations
2034 Solved in 14 iterations
2035 Solved in 15 iterations
2036 Solved in 14 iterations
2037 Solved in 14 iterations
2038 Solved in 14 iterations
2039 Solved in 7 iterations
2040 Solved in 7 iterations
AGO solved  


## And the simulation result is also fine. 
Here the percent difference is displayed

In [12]:
mago['AGOGGEXPCAPTCN AGONYGDPMKTPCN AGOGGDBTTOTLCN AGOBNCABFUNDCD'].difpctlevel.rename()

      Capital Expenditure   Nominal GDP     Govt Debt  Current account
2016         0.0000000000 -0.0000000000 -0.0000000000    -0.0000000000
2017         0.0000000000 -0.0000000000 -0.0000000000     0.0000000000
2018        -0.0000000000 -0.0000000000 -0.0000000000    -0.0000000000
2019         0.0000000000 -0.0000000000 -0.0000000000    -0.0000000000
2020         0.0000000409 -0.0000000309 -0.0000000005     0.0000000008
2021         0.0000000333 -0.0000000077  0.0000000029     0.0000000932
2022        -0.0000001683  0.0000000047  0.0000000064     0.0000000431
2023        -0.0000001123  0.0000000462  0.0000000100     0.0000000439
2024         0.0000000202  0.0000002660  0.0000000103    -0.0000000030
2025         0.0000000121  0.0000001002 -0.0000000155    -0.0000000807
2026         0.0000002434  0.0000001346  0.0000000060    -0.0000000924
2027         0.0000000705  0.0000000545 -0.0000000079    -0.0000000838
2028         0.0000000042 -0.0000001176 -0.0000000320    -0.0000001683
2029  

# One variable is investigated 
AGONVGDPOTHRCN shows a difference of 355. Although a small percent lets look at it.

**all_about_ago** has a lot of content. One is .all_frml_dict. This is a dict with en entry for each endogenous variable and a value which is an 
instances of a dataclass called **Normalized_frml**. <br>
Each Normalized_frml contains 
the different phases a frml goes through from eviews to modelflow equation. It also contains the equations for add factors, fitted variables and calculation of add factors.

For AGONVGDPOTHRCN which is an identity with no frills the Normalized_frml looks like this: 

In [15]:
all_about_ago.all_frml_dict['AGONVGDPOTHRCN'].fprint


Endo_var        : AGONVGDPOTHRCN
Original        : AGONVGDPOTHRCN=AGONVGDPOTHRKN*AGONVGDPOTHRXN
Preprocessed    : AGONVGDPOTHRCN=AGONVGDPOTHRKN*AGONVGDPOTHRXN
Normalized      : AGONVGDPOTHRCN = AGONVGDPOTHRKN*AGONVGDPOTHRXN
Eviews          : @IDENTITY agonvgdpothrcn  = agonvgdpothrkn  * agonvgdpothrxn


How can such a simple frml give different results in eviews and in modelflow. Lets look at the data: 

The base dataframne contains the values imported from eviews

The lastdf contains the calculated variables. 

The input values are the same in eviews and modelflow. If one does the calulation by hand the modelflow results are right. <br>
One explanation could be that eviews truncates some of the values to single precision  when calculating some values. 

In [16]:
with mago.set_smpl(2057,2059):
    print(mago.AGONVGDPOTHRCN.show)                        

Endogeneous: AGONVGDPOTHRCN:  
Formular: FRML  <> AGONVGDPOTHRCN = AGONVGDPOTHRKN*AGONVGDPOTHRXN $

AGONVGDPOTHRCN: 
AGONVGDPOTHRKN: 
AGONVGDPOTHRXN: 

Values : 
                       2057                   2058                   2059
Base 63963956012.8213348389 69151728515.2461853027 74736766907.8641052246
Last 63963956012.8213348389 69151728515.2461853027 74736766907.8641052246
Diff           0.0000000000           0.0000000000           0.0000000000

Input last run: 
                              2057               2058               2059
AGONVGDPOTHRKN 7016583.2910357974 7461688.5852832915 7934692.3573708227
AGONVGDPOTHRXN    9116.1115678822    9267.5709885311    9418.9873010611

Input base run: 
                              2057               2058               2059
AGONVGDPOTHRKN 7016583.2910357974 7461688.5852832915 7934692.3573708227
AGONVGDPOTHRXN    9116.1115678822    9267.5709885311    9418.9873010611

Difference for input variables: 
                        2057         2

In [18]:
with mago.set_smpl(2020,2020):
    print(mago.AGOGGEXPCAPTCN.show)                        

Endogeneous: AGOGGEXPCAPTCN: Capital Expenditure 
Formular: FRML <Z,EXO> AGOGGEXPCAPTCN = (0.15*AGOGGEXPCAPTCN(-1)+(1-0.15)*((1-((AGOGGEXPGNFSCN_VALUE_2019/(AGOGGEXPTOTLCN_VALUE_2019))))*(1+-0.15)*(AGOGGREVTOTLCN-AGOGGEXPINTPCN))-1*(AGODISPREPCN)-1950858.15761458*DURING_2018 + AGOGGEXPCAPTCN_A)* (1-AGOGGEXPCAPTCN_D)+ AGOGGEXPCAPTCN_X*AGOGGEXPCAPTCN_D  $

AGOGGEXPCAPTCN           : Capital Expenditure
AGODISPREPCN             : 
AGOGGEXPCAPTCN_A         : Add factor:Capital Expenditure
AGOGGEXPCAPTCN_D         : Exo dummy:Capital Expenditure
AGOGGEXPCAPTCN_X         : Exo value:Capital Expenditure
AGOGGEXPGNFSCN_VALUE_2019: 
AGOGGEXPINTPCN           : Interest Payments
AGOGGEXPTOTLCN_VALUE_2019: 
AGOGGREVTOTLCN           : Total Revenue
DURING_2018              : 

Values : 
                   2020
Base 2739923.9490547292
Last 2739924.0612037797
Diff       0.1121490505

Input last run: 
                                         2020
AGODISPREPCN                    0.0000000000
AGOGGEXPCA

## Look at a stochastic variable 
Here the equations undergo more phases 

In [19]:
all_about_ago.all_frml_dict['AGONVINDMINGKN'].fprint


Endo_var        : AGONVINDMINGKN
Original        : DLOG(AGONVINDMINGKN)=-0.3*(LOG(AGONVINDMINGKN(-1))-LOG(AGOSMINDMINGKN(-1)))+0.192238955968179+DLOG(AGOSMINDMINGKN)+0.0064190961088093*T_LR
Preprocessed    : ((LOG(AGONVINDMINGKN))-(LOG(AGONVINDMINGKN(-1))))=-0.3*(LOG(AGONVINDMINGKN(-1))-LOG(AGOSMINDMINGKN(-1)))+0.192238955968179+((LOG(AGOSMINDMINGKN))-(LOG(AGOSMINDMINGKN(-1))))+0.0064190961088093*T_LR
Normalized      : AGONVINDMINGKN = (AGONVINDMINGKN(-1)*EXP(-AGONVINDMINGKN_A+ (-0.3*(LOG(AGONVINDMINGKN(-1))-LOG(AGOSMINDMINGKN(-1)))+0.192238955968179+((LOG(AGOSMINDMINGKN))-(LOG(AGOSMINDMINGKN(-1))))+0.0064190961088093*T_LR) )) * (1-AGONVINDMINGKN_D)+ AGONVINDMINGKN_X*AGONVINDMINGKN_D 
Calc_add_factor : AGONVINDMINGKN_A =  ((-0.3*(LOG(AGONVINDMINGKN(-1))-LOG(AGOSMINDMINGKN(-1)))+0.192238955968179+((LOG(AGOSMINDMINGKN))-(LOG(AGOSMINDMINGKN(-1))))+0.0064190961088093*T_LR)) -LOG(AGONVINDMINGKN)+LOG(AGONVINDMINGKN(-1))
Fitted          : AGONVINDMINGKN_FITTED = AGONVINDMINGKN(-1)*EXP( (-0.3

## Look a all the modelflow frmls
Notice after the "original" model the equations for the "fitted" values have been added. <br>
Also in the end of the listing the specification of the model which calculates the add factors if a variable is fixed. When processing the equations the ```model``` class will process this this model separately and create a model instance 
which is used to calculate add factors in case 

In [ ]:
# print(mago.equations)

In [20]:
with mago.set_smpl(2020,2023):
    print(mago.AGOGGEXPCAPTCN.show)                        

Endogeneous: AGOGGEXPCAPTCN: Capital Expenditure 
Formular: FRML <Z,EXO> AGOGGEXPCAPTCN = (0.15*AGOGGEXPCAPTCN(-1)+(1-0.15)*((1-((AGOGGEXPGNFSCN_VALUE_2019/(AGOGGEXPTOTLCN_VALUE_2019))))*(1+-0.15)*(AGOGGREVTOTLCN-AGOGGEXPINTPCN))-1*(AGODISPREPCN)-1950858.15761458*DURING_2018 + AGOGGEXPCAPTCN_A)* (1-AGOGGEXPCAPTCN_D)+ AGOGGEXPCAPTCN_X*AGOGGEXPCAPTCN_D  $

AGOGGEXPCAPTCN           : Capital Expenditure
AGODISPREPCN             : 
AGOGGEXPCAPTCN_A         : Add factor:Capital Expenditure
AGOGGEXPCAPTCN_D         : Exo dummy:Capital Expenditure
AGOGGEXPCAPTCN_X         : Exo value:Capital Expenditure
AGOGGEXPGNFSCN_VALUE_2019: 
AGOGGEXPINTPCN           : Interest Payments
AGOGGEXPTOTLCN_VALUE_2019: 
AGOGGREVTOTLCN           : Total Revenue
DURING_2018              : 

Values : 
                   2020               2021               2022               2023
Base 2739923.9490547292 3246400.8487510146 3918275.2782179881 7285398.1806929344
Last 2739924.0612037797 3246400.9568246650 3918274.61